## ENGRI 1120: High-Fructose Corn Syrup (HFCS) Process Simulation

### Introduction

High Fructose Corn Syrup (HFCS) is a common sweetener in many food products. As a sweetener, HFCS is often compared to granulated table sugar (sucrose is made up of one molecule of glucose and one molecule of fructose); HFCS is easier to handle and cheaper than granulated table sugar. HFCS comes in two varieties, HFCS-42 and HFCS-55. HFCS-42 and HFCS-55 refer to dry-weight fructose compositions of 42% and 55%, respectively, the rest being glucose. HFCS-42 is mainly used for processed foods and breakfast cereals, whereas HFCS-55 is used chiefly for producing soft drinks.

### Manufacturing Process
Corn is milled to extract corn starch, and an "acid-enzyme" process is used, in which the corn-starch solution is acidified to break up the existing carbohydrates. High-temperature enzymes are added to metabolize the starch further and convert the resulting sugars to fructose.
The first enzyme, alpha-amylase, breaks the long starch chains down into shorter sugar chains (oligosaccharides). 
Glucoamylase, a second enzyme, converts the oligosaccharides to glucose (a common six-carbon sugar for biotechnology applications). The resulting solution is filtered to remove protein, then using activated carbon, and then demineralized using ion-exchange resins. 

The purified solution is then run over immobilized xylose isomerase, which converts the sugars to ~50–52% glucose with some unconverted oligosaccharides and 42% fructose (HFCS-42), and again demineralized and again purified using activated carbon. Some is processed into HFCS-90 by liquid chromatography and then mixed with HFCS-42 to form HFCS-55. The enzymes used in the process are made by microbial fermentation. 

<img src="figs/Fig-HCFS-Reactor.png" style="width:50%">

### Model and Assumptions
Let's model the immobilized xylose isomerase step of the HCFS-42 process. Consider a reactor in which enzyme $E$ catalyzes the conversion of some substrate $S$ (starting material) to a product $P$ at a rate $\hat{r}_{1}$ (units: mmol/L-time). Enzyme $E$, which is not stable, degrades in the reactor at rate $r_{2}$ (units: mmol/L-time). Susbtrate $S$ is introduced into the reactor in stream 1, while enzyme $E$ is introduced in stream 2. Lastly, unreacted subrate $S$, enzyme $E$ and product $P$ leave the reactor in stream 3.

Let the concentrations of $S$ be given by $C_{1}$ (units: mmol/L), the enzyme $E$ by $C_{2}$ (units: mmol/L) and the product $P$ by $C_{3}$ (units: mmol/L). Then, the kinetic expression for reaction $\hat{r}_{1}$ is given by:

$$\hat{r}_{1} = k_{cat}E\left(\frac{C_{1}}{K+C_{1}}\right)$$

where $k_{cat}$ denotes the turnover number (catalytic rate constant with units 1/time) for the enzyme $E$ and substate $S$, and $K$ denotes the saturation constant for enzyme $E$ and substrate $S$ (units: concentration). The rate of degradation of enzyme $E$ is assumed to be first-order with rate:

$$\hat{r}_{2} = k_{d}E$$

where $k_{d}$ denotes the degradation constant for enzyme $E$ (units: 1/time). 

#### Assumptions

* Reactor is at steady-state and well-mixed
* Reactor is at a constant T, P and V
* Density of stream 1, 2 and 3 is constant and equal to water at T and P. 

#### Problem setup
The volume of the reactor $V$ = 30$\mu$L. Stream 1 has a volumetric flow rate $F_{1}$ = 10$\mu$L/h and Stream 2 has a volumetric flow rate $F_{2}$ = 5$\mu$L/h. The concentration(s) of susbstrate in the stream 1 is $C_{1,1}$ = 10 mmol/L, enzyme $E$ in stream 2 is $C_{2,2}$ = 2.0 mmol/L. The turnover number of enzyme $E$ is give by $k_{cat}$ = 16.2 s$^{-1}$, the saturation coefficient $K$ = 5 mmol/L and degradation constant $k_{d}$ = 0.1 h$^{-1}$. 

### Example setup

In [1]:
import Pkg; Pkg.activate("."); Pkg.resolve(); Pkg.instantiate();

  Activating project at `~/Desktop/julia_work/ENGRI-1120-IntroToChemE-Example-Notebooks/notebooks-jupyter/ENGRI-1120-HFCS-Example`
  No Changes to `~/Desktop/julia_work/ENGRI-1120-IntroToChemE-Example-Notebooks/notebooks-jupyter/ENGRI-1120-HFCS-Example/Project.toml`
  No Changes to `~/Desktop/julia_work/ENGRI-1120-IntroToChemE-Example-Notebooks/notebooks-jupyter/ENGRI-1120-HFCS-Example/Manifest.toml`


In [2]:
include("ENGRI-1120-HFCS-CodeLib.jl")

testfunc (generic function with 1 method)

In [3]:
# load req packages -
using PrettyTables
using LinearAlgebra
using Optim

# setup paths -
const _ROOT = pwd();
const _PATH_TO_DATA = joinpath(_ROOT, "data");

### Solve steady-state concetration balances for exit composition

In [10]:
# setup model parameters -
model = HFCSParameterModel();
model.k₁ = 16.2; # units: 1/h
model.k₂ = 0.1; # units: 1/h
model.Kₘ = 5.0; # units: mmol/L

# Compute the dilution rates 
# We have three dilution rates Dₛ s = 1,2,3
V = 30.0*(1e-6);  # units: L
V̇₁ = 10.0*(1e-6); # units: L/h
V̇₂ = 5.0*(1e-6); # units: L/h
V̇₃ = (V̇₁+ V̇₂);   # units: L/h
model.D₁ = (1/V)*V̇₁;   # units: 1/h
model.D₂ = (1/V)*V̇₂;   # units: 1/h
model.D₃ = (1/V)*V̇₃;   # units: 1/h

# Setup the stoichiometric matrix for this system
# SM is a 3 x 2 matrix (species = 3, reactions = 2)
SM = zeros(3,2);
SM[1,1] = -1.0;
SM[2,2] = -1.0;
SM[3,1] = 1.0;
model.SM = SM;

# Setup the concentration matrix
# CM is a 3 x 3 matrix (species x streams)
CM = zeros(3,3);
CM[1,1] = 100.0; 
CM[2,2] = 1.0;
model.CM = CM;

In [12]:
# setup calculation -

# Use the feed input as a starting point guess for the search 
xinitial = [CM[1,1]*model.D₁, CM[2,2]*model.D₂, 1e-8]

# Setup the objective function (this is the function that we will minimize)
OF(x) = objfunc(x, model)

# setup bounds -
L = [0.0, 0.0, 0.0];  # lower bound is zero (concentration ≥ 0)
U = [Inf, Inf, Inf];  # upper bound is Inf (concentration ≤ ∞)
    
# call the optimizer -
opt_result = optimize(OF, L, U, xinitial, Fminbox(NelderMead()))

 * Status: success

 * Candidate solution
    Final objective value:     2.451237e-09

 * Found with
    Algorithm:     Fminbox with Nelder-Mead

 * Convergence measures
    |x - x'|               = 0.00e+00 ≤ 0.0e+00
    |x - x'|/|x'|          = 0.00e+00 ≤ 0.0e+00
    |f(x) - f(x')|         = 0.00e+00 ≤ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 0.00e+00 ≤ 0.0e+00
    |g(x)|                 = 3.37e+02 ≰ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    5
    f(x) calls:    776
    ∇f(x) calls:   1


In [6]:
nm_soln = Optim.minimizer(opt_result)

3-element Vector{Float64}:
 58.37637077944712
  0.27779062035965596
  8.290251885577698

In [8]:
error_result = objfunc(nm_soln, model)

2.4512368177672826e-9

In [13]:
# put the answer in the CM -
CM[1,3] = nm_soln[1];
CM[2,3] = nm_soln[2];
CM[3,3] = nm_soln[3];

In [17]:
# put the D's in a vector -
D_vector = [
    model.D₁ ;
    model.D₂ ; 
    model.D₃ ;
];

### Stream composition table

In [27]:
# setup table -
number_of_streams = 3;
state_table_data_array = Array{Any,2}(undef, (number_of_streams+1), 4);

# populate the table -
for s ∈ 1:number_of_streams
    
    # stream
    state_table_data_array[s,1] = s;
    state_table_data_array[s,2] = CM[1,s]*D_vector[s];
    state_table_data_array[s,3] = CM[2,s]*D_vector[s];
    state_table_data_array[s,4] = CM[3,s]*D_vector[s];
end

# total -
state_table_data_array[4,1] = "Change"
state_table_data_array[4,2] = -1*(CM[1,1]*D_vector[1] - CM[1,3]*D_vector[3])
state_table_data_array[4,3] = -1*(CM[2,2]*D_vector[2] - CM[2,3]*D_vector[3])
state_table_data_array[4,4] = CM[3,3]*D_vector[3]



# setup a header -
header_data = (["Stream s", "ṅₛ,1 (mol/time)", "ṅₛ,2 (mol/time)", "ṅₛ,3 (mol/time)"])

# draw a table -
pretty_table(state_table_data_array; header=header_data)

┌──────────┬─────────────────┬─────────────────┬─────────────────┐
│ Stream s │ ṅₛ,1 (mol/time) │ ṅₛ,2 (mol/time) │ ṅₛ,3 (mol/time) │
├──────────┼─────────────────┼─────────────────┼─────────────────┤
│        1 │         33.3333 │             0.0 │             0.0 │
│        2 │             0.0 │        0.166667 │             0.0 │
│        3 │         29.1882 │        0.138895 │         4.14513 │
│   Change │        -4.14515 │      -0.0277714 │         4.14513 │
└──────────┴─────────────────┴─────────────────┴─────────────────┘
